# Type Inference and Program Synthesis

This notebook demonstrates how to encode a type system in SGGSLog and use it for:

1. **Type checking**: Given a term and type, verify the term has that type
2. **Type inference**: Given a term, find its type
3. **Program synthesis**: Given a type, find a term that has that type

The third direction is remarkable: it's essentially proof search via the Curry-Howard correspondence.
Finding a program of type `A -> A` is the same as finding a proof of the proposition A implies A.

## Lambda Calculus Syntax

We encode the simply-typed lambda calculus as compound terms:

**Terms:**
- `(var X)` - a variable reference
- `(lam X Body)` - lambda abstraction (function)
- `(app F Arg)` - function application

**Types:**
- `nat`, `bool`, etc. - base types
- `(arrow A B)` - function type A -> B

**Typing contexts:**
- `nil` - empty context
- `(cons (bind X T) Env)` - context extended with variable X having type T

In [ ]:
:closed name
:closed lookup
:closed types


In [ ]:
name x
name y
name f
name g


## Context Lookup

First, we define how to look up a variable's type in a context.
The lookup succeeds if we find a binding for the variable.

In [ ]:
lookup (cons (bind X T) Env) X T

In [ ]:
lookup Env X T -> lookup (cons (bind Y S) Env) X T

## Typing Rules

Now we encode the standard typing rules for the simply-typed lambda calculus:

1. **VAR**: A variable has whatever type the context says
2. **LAM**: A lambda `\x.body` has type `A -> B` if body has type B when x has type A
3. **APP**: An application `(f arg)` has type B if f has type `A -> B` and arg has type A

In [ ]:
name X & lookup Env X T -> types Env (var X) T

In [ ]:
name X & types (cons (bind X A) Env) Body B -> types Env (lam X Body) (arrow A B)

In [ ]:
types Env F (arrow A B) & types Env Arg A -> types Env (app F Arg) B

## Type Checking

With these rules, we can check if a term has a specific type.

**Example**: Does `\x.x` (the identity function) have type `nat -> nat`?

In [ ]:
?- types nil (lam x (var x)) (arrow nat nat)

**Example**: Does `\x.\y.x` (the K combinator) have type `nat -> bool -> nat`?

In [ ]:
?- types nil (lam x (lam y (var x))) (arrow nat (arrow bool nat))

## Type Inference

The same rules work for type inference: leave the type as a variable.

**Example**: What type does the identity function have?

In [ ]:
?- types nil (lam x (var x)) T

**Example**: What type does `\x.\y.x` have?

In [ ]:
?- types nil (lam x (lam y (var x))) T

**Example**: What type does applying identity to itself have? (`(\x.x)(\x.x)` )


## Program Synthesis

Now for the most impressive direction: given a type, find a program!

This is proof search via Curry-Howard. The type is a proposition, and we're looking for a proof (program) that inhabits it.

**Example**: Find a term of type `A -> A`


In [ ]:
?- types nil Term (arrow A A)

**Example**: Find a term of type `A -> B -> A` (the K combinator)

In [ ]:
?- types nil Term (arrow A (arrow B A))

**Example**: Find a term of type `(A -> B -> C) -> (A -> B) -> A -> C` (the S combinator)

This is a more complex synthesis task.


## Function Composition

**Constrained example**: To avoid schematic answers, we add a sketch and fix distinct
base types `a`, `b`, `c`. This keeps the search focused while still using the same
`types` relation.


In [ ]:
?- types nil (lam f (lam g (lam x (app (var g) (app (var f) (var x)))))) (arrow (arrow a b) (arrow (arrow b c) (arrow a c)))


## Summary

We've demonstrated three uses of the same typing relation:

1. **Type checking**: `types nil term type` with both term and type ground
2. **Type inference**: `types nil term T` with T as a variable
3. **Program synthesis**: `types nil Term type` with Term as a variable

The synthesis direction is essentially automated theorem proving via the Curry-Howard correspondence:
- Types are propositions
- Programs are proofs
- Finding a well-typed term is finding a proof

This bidirectionality is a hallmark of logic programming - one relation, many uses.